In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
import pyperclip
import time
import re
import pandas as pd
import datetime
import csv

In [2]:
the_csv = '/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/Actual - CSV - Input - Algorithm/actual.csv'

In [3]:
while True:
    try:
        ### Start Script ###
        # Launch Firefox browser
        browser = webdriver.Firefox() # Now you can control this browser by calling methods on browser

        # Get method - send the browser to URL
        browser.get('https://sigma.icr.usp.br/login.aspx?ReturnUrl=%2frestrito%2fdefault.aspx')

        # Get element object using XPATH
        elemSearch = browser.find_element(By.XPATH, '//*[@id="txtUsuario"]')

        # Now you can click the element using the click method
        elemSearch.click()

        # Now you can type text in the element
        elemSearch.clear() # Clear if something is already typed
        elemSearch.send_keys('username')

        # Click in password field and type the password
        elemPassword = browser.find_element(By.XPATH, '//*[@id="txtSenha"]')
        elemPassword.click()
        elemPassword.send_keys('password')

        # Click enviar
        elemEnviar = browser.find_element(By.XPATH, '//*[@id="btnLogin"]')
        elemEnviar.click()

        # Click ASSISTENCIAL
        elemAssistencial = WebDriverWait(browser,10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeRotinast4"]')))
        elemAssistencial.click()

        # Click Consulta Exames
        elemConExam = browser.find_element(By.XPATH, '//*[@id="treeRotinast7"]')
        elemConExam.click()

        # Click Exames
        elemExames = browser.find_element(By.XPATH, '//*[@id="treeRotinast9"]')
        elemExames.click()

        # Switch to iFrame
        WebDriverWait(browser, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))

        # Input matricula
        elemSearchMatricula = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ucPacientes1_txtBusca"]')))
        elemSearchMatricula.click()
        elemSearchMatricula.send_keys(' ')

        # Click consultar
        elemConsultar = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ucPacientes1_btnBuscaPaciente"]')))
        elemConsultar.click()
        time.sleep(5)

        # Switch to 30 registros
        elemRegistros = WebDriverWait(browser,30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="cboPaginas"]')))
        elemRegistros.click()

        elem30Registros = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[1]/table/tbody/tr[1]/td/select[2]/option[3]')
        elem30Registros.click()

        # Set date
        elemDate = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="txtDtInicial"]')))
        elemDate.click()
        elemDateYear = browser.find_element(By.XPATH, '/html/body/div/div/div/select[2]')
        elemDateYear.click()

        elemDateYear2012 = browser.find_element(By.XPATH, '/html/body/div/div/div/select[2]/option[1]')
        elemDateYear2012.click()

        elemDateMonth = browser.find_element(By.XPATH, '/html/body/div/div/div/select[1]')
        elemDateMonth.click()

        elemDateMonthJan = browser.find_element(By.XPATH, '/html/body/div/div/div/select[1]/option[1]')
        elemDateMonthJan.click()

        elemDateMonthDia = browser.find_element(By.XPATH, '/html/body/div/table/tbody/tr[1]/td[1]/a')
        elemDateMonthDia.click()

        with open(the_csv, 'r') as f:
            new_df = pd.read_csv(f, delimiter=";")
            new_df = new_df.drop('Unnamed: 0', axis=1)

        # Get matricula and outcome date
        for index, row in new_df.iterrows():
            matricula = row['rghc'].rstrip()
            time.sleep(2)
            elemInsertMatricula = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ucPacientes1_txtBusca"]')))
            #elemInsertMatricula.click()
            elemInsertMatricula.clear()
            elemInsertMatricula.send_keys(matricula)

            elemRefresh = browser.find_element(By.XPATH, '//*[@id="ucPacientes1_btnBuscaPaciente"]')
            elemRefresh.click()
            time.sleep(60)

            new_string = ''
            
            WebDriverWait(browser, 120).until(EC.invisibility_of_element((By.XPATH, '//*[@id="bgloading"]')))
            time.sleep(1)
            WebDriverWait(browser, 120).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]')))

            df = pd.DataFrame()
            df_echo = pd.DataFrame()
            df_usg = pd.DataFrame()

            try:
                # If multiple patients with same name
                WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[4]/table/tbody/tr[1]')))
                
                #with open('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/erro.csv', 'a') as f:
                    #fieldnames = ['Nome', 'Data_Nasc']
                    #writer = csv.DictWriter(f, fieldnames=fieldnames)
                    #writer.writerow({'Nome': matricula, 'Data_Nasc':row['Data_alta']})
                    
            except TimeoutException:
                pass

                try:
                    # If no patient found
                    WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, '//td[text()="Nenhum registro encontrado"]')))
                    #with open('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/erro.csv', 'a') as f:
                        #fieldnames = ['Nome', 'Data_Nasc']
                        #writer = csv.DictWriter(f, fieldnames=fieldnames)
                        #writer.writerow({'Nome': matricula, 'Data_Nasc':row['Data_alta']})
                    pass

                except TimeoutException:
                    # Patient found
                    elemRg = WebDriverWait(browser,2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="lblDadosPaci"]'))).text
                    NOME_PACIENTE = re.findall(r'-(.*)-', elemRg)
                    nome_paciente = ''.join(NOME_PACIENTE)
                    elemRg = elemRg[:11]

                    try:
                        elemQtdDePagina = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[4]/table/tbody/tr[1]/td/table/tbody/tr')
                        qtdDePagina = elemQtdDePagina.text
                        list_quantidade = qtdDePagina.split(' ')

                        if list_quantidade[-1] == 'Ultima':
                            elemUltima = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[4]/table/tbody/tr[1]/td/table/tbody/tr/td[12]')
                            elemUltima.click()
                            time.sleep(1)
                            elemUltimaPagina = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[4]/table/tbody/tr[1]/td/table/tbody/tr')
                            ultimaPagina = elemUltimaPagina.text.split(' ')[-1]
                            primeiraPagina = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[4]/table/tbody/tr[1]/td/table/tbody/tr/td[1]')
                            primeiraPagina.click()
                            time.sleep(1)

                        else:
                            ultimaPagina = list_quantidade[-1]
                            time.sleep(1)

                        for pagina in range(2,int(ultimaPagina)+1):
                            ### FIND ECHO 1 ###
                            try:
                                elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/preceding-sibling::td[2]')
                                time.sleep(3)

                                if browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                    list_echo = [elemRg]
                                    string_echo = ''
                                    elemFindEcho.click()
                                    time.sleep(5)

                                    try:
                                        WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                        browser.switch_to.window(browser.window_handles[-1])
                                        time.sleep(3)

                                        elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                        time.sleep(2)
                                        elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')
                                        
                                        string_echo = pyperclip.paste()

                                        ### REGEX ###

                                        DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame = ''.join(DATA_EXAME)
                                        list_echo.append(data_exame[:10])

                                        DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame_2 = ''.join(DATA_EXAME_2)
                                        list_echo.append(data_exame_2[:10])

                                        PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                        pca_tamanho = ''.join(PCA_TAMANHO)
                                        list_echo.append(pca_tamanho)

                                        PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                        pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                        list_echo.append(pca_tamanho2)

                                        DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                        diametro_aorta = ''.join(DIAMETRO_AORTA)
                                        list_echo.append(diametro_aorta)

                                        DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                        list_echo.append(diametro_aorta_2)

                                        DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                        list_echo.append(diametro_aorta_3)

                                        DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                        diametro_ae = ''.join(DIAMETRO_AE)
                                        list_echo.append(diametro_ae)

                                        DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                        list_echo.append(diametro_ae_2)

                                        DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                        list_echo.append(diametro_ae_3)

                                        ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                        list_echo.append(encurtamento_ve)

                                        FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        feve = ''.join(FEVE)
                                        list_echo.append(feve)

                                        CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                        civ_1 = ''.join(CIV_1)
                                        list_echo.append(civ_1)

                                        VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                        vti_p = ''.join(VTI_P)
                                        list_echo.append(vti_p)

                                        PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                        psap = ''.join(PSAP)
                                        list_echo.append(psap)

                                        new_df_echo = pd.DataFrame([list_echo])
                                        df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                        time.sleep(1)
                                        
                                        # Close tab
                                        browser.close()
                                        time.sleep(1)
                                        browser.switch_to.window(browser.window_handles[0])
                                        WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                    except TimeoutException:
                                        pass
                                
                                elif browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Laudado':
                                    list_echo = [elemRg]
                                    string_echo = ''
                                    elemFindEcho.click()
                                    time.sleep(5)

                                    try:
                                        WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))
                                        browser.switch_to.window(browser.window_handles[-1])
                                        time.sleep(3)

                                        elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                        time.sleep(2)
                                        elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                        string_echo = pyperclip.paste()

                                        ### REGEX ###
                                        DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame = ''.join(DATA_EXAME)
                                        list_echo.append(data_exame[:10])

                                        DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame_2 = ''.join(DATA_EXAME_2)
                                        list_echo.append(data_exame_2[:10])

                                        PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                        pca_tamanho = ''.join(PCA_TAMANHO)
                                        list_echo.append(pca_tamanho)

                                        PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                        pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                        list_echo.append(pca_tamanho2)

                                        DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                        diametro_aorta = ''.join(DIAMETRO_AORTA)
                                        list_echo.append(diametro_aorta)

                                        DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                        list_echo.append(diametro_aorta_2)

                                        DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                        list_echo.append(diametro_aorta_3)

                                        DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                        diametro_ae = ''.join(DIAMETRO_AE)
                                        list_echo.append(diametro_ae)

                                        DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                        list_echo.append(diametro_ae_2)

                                        DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                        list_echo.append(diametro_ae_3)

                                        ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                        list_echo.append(encurtamento_ve)

                                        FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        feve = ''.join(FEVE)
                                        list_echo.append(feve)

                                        CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                        civ_1 = ''.join(CIV_1)
                                        list_echo.append(civ_1)

                                        VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                        vti_p = ''.join(VTI_P)
                                        list_echo.append(vti_p)

                                        PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                        psap = ''.join(PSAP)
                                        list_echo.append(psap)
                                        
                                        new_df_echo = pd.DataFrame([list_echo])
                                        df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                        
                                        # Close tab
                                        browser.close()
                                        time.sleep(1)
                                        browser.switch_to.window(browser.window_handles[0])
                                        WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                    except TimeoutException:
                                        pass
                                else:
                                    pass

                            except NoSuchElementException:
                                pass

                            ### FIND ECHO 2 ###
                            try:
                                elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/preceding-sibling::td[2]')
                                time.sleep(3)

                                if browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                    list_echo = [elemRg]
                                    string_echo = ''
                                    elemFindEcho.click()
                                    time.sleep(5)

                                    try:
                                        WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                        browser.switch_to.window(browser.window_handles[-1])
                                        time.sleep(3)
                                        
                                        elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                        time.sleep(2)
                                        elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                        string_echo = pyperclip.paste()

                                        ### REGEX ###

                                        DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame = ''.join(DATA_EXAME)
                                        list_echo.append(data_exame[:10])

                                        DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                        data_exame_2 = ''.join(DATA_EXAME_2)
                                        list_echo.append(data_exame_2[:10])

                                        PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                        pca_tamanho = ''.join(PCA_TAMANHO)
                                        list_echo.append(pca_tamanho)

                                        PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                        pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                        list_echo.append(pca_tamanho2)

                                        DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                        diametro_aorta = ''.join(DIAMETRO_AORTA)
                                        list_echo.append(diametro_aorta)

                                        DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                        list_echo.append(diametro_aorta_2)

                                        DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                        diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                        list_echo.append(diametro_aorta_3)

                                        DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                        diametro_ae = ''.join(DIAMETRO_AE)
                                        list_echo.append(diametro_ae)

                                        DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                        list_echo.append(diametro_ae_2)

                                        DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                        diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                        list_echo.append(diametro_ae_3)

                                        ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                        list_echo.append(encurtamento_ve)

                                        FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                        feve = ''.join(FEVE)
                                        list_echo.append(feve)

                                        CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                        civ_1 = ''.join(CIV_1)
                                        list_echo.append(civ_1)

                                        VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                        vti_p = ''.join(VTI_P)
                                        list_echo.append(vti_p)

                                        PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                        psap = ''.join(PSAP)
                                        list_echo.append(psap)
                                    
                                        new_df_echo = pd.DataFrame([list_echo])
                                        df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)

                                        # Close tab
                                        browser.close()
                                        time.sleep(1)
                                        browser.switch_to.window(browser.window_handles[0])
                                        WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                    except TimeoutException:
                                        pass

                                else:
                                    pass

                            except NoSuchElementException:
                                pass


                            ### FIND USG 1 ###
                            try:
                                elemFindUSG = browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/preceding-sibling::td[2]')

                                if browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/..').text.split(' ')[-1] == 'Liberado':
                                    list_imagem = [elemRg]
                                    string_image = ''
                                    elemFindUSG.click()
                                    time.sleep(5)

                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)
                                    
                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_image = pyperclip.paste()

                                    ### REGEX ###
                                    DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_imagem.append(data_exame)

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_imagem.append(data_exame_2)

                                    HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                    hemorragia = ''.join(HEMORRAGIA)
                                    list_imagem.append(hemorragia)

                                    new_df_usg = pd.DataFrame([list_imagem])
                                    df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))

                                else:
                                    pass
                                
                            except NoSuchElementException:
                                pass

                            ### FIND USG 2 ###
                            try:
                                elemFindUSG_2 = browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/preceding-sibling::td[2]')

                                if browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/..').text.split(' ')[-1] == 'Liberado':
                                    list_imagem = [elemRg]
                                    string_image = ''
                                    
                                    elemFindUSG_2.click()
                                    time.sleep(5)
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_image = pyperclip.paste()

                                    ### REGEX ###
                                    DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_imagem.append(data_exame)

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_imagem.append(data_exame_2)

                                    HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                    hemorragia = ''.join(HEMORRAGIA)
                                    list_imagem.append(hemorragia)

                                    new_df_usg = pd.DataFrame([list_imagem])
                                    df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                else:
                                    pass
                                
                            except NoSuchElementException:
                                pass

                            ### FIND USG 3 ###
                            try:
                                elemFindUSG_3 = browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/preceding-sibling::td[3]')

                                if browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/..').text.split(' ')[-1] == 'Laudado':
                                    list_imagem = [elemRg]
                                    string_image = ''
                                    
                                    elemFindUSG_3.click()
                                    time.sleep(5)
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_image = pyperclip.paste()

                                    ### REGEX ###
                                    DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_imagem.append(data_exame)

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_imagem.append(data_exame_2)

                                    HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                    hemorragia = ''.join(HEMORRAGIA)
                                    list_imagem.append(hemorragia)

                                    new_df_usg = pd.DataFrame([list_imagem])
                                    df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                else:
                                    pass
                            except NoSuchElementException:
                                pass

                            elemSelectExams = WebDriverWait(browser,120).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[1]/table/tbody/tr/td/font/b/u')))
                            elemSelectExams.click()
                            time.sleep(1)

                            try:
                                elemExhibitExames = WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[1]/input')))
                                elemExhibitExames.click()
                                time.sleep(2)

                                WebDriverWait(browser, 60).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="frameRelatorio"]')))
                                                
                                elemBody = WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="resultHTML"]')))
                                elemBody.click()

                                elemBody.send_keys(Keys.COMMAND + 'a')
                                elemBody.send_keys(Keys.COMMAND + 'c')

                                new_string = new_string + '\n' + pyperclip.paste()

                                # Voltar
                                elemVoltar = WebDriverWait(browser,60).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnVoltarLista"]')))
                                elemVoltar.click()
                                time.sleep(2)

                                # Limpar
                                browser.switch_to.default_content()
                                WebDriverWait(browser,3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                elemLimpar = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[3]')))
                                elemLimpar.click()
                            
                            except TimeoutException:
                                pass
                            pagina = str(pagina)
                            browser.find_element(By.CSS_SELECTOR, "a[href*='Page${}']".format(pagina)).click()
                            
                        # Last Page
                        ### FIND ECHO 1 ###
                        try:
                            elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/preceding-sibling::td[2]')
                            time.sleep(3)

                            if browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(5)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))
                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')
                                    
                                    string_echo = pyperclip.paste()

                                    ### REGEX ###

                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)

                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                    
                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass
                            
                            elif browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Laudado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(8)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_echo = pyperclip.paste()

                                    ### REGEX ###
                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)
                                    
                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                    
                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass
                            else:
                                pass

                        except NoSuchElementException:
                            pass

                        ### FIND ECHO 2 ###
                        try:
                            elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/preceding-sibling::td[2]')
                            time.sleep(3)

                            if browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(5)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)
                                    
                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_echo = pyperclip.paste()

                                    ### REGEX ###

                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)
                                
                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)

                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass

                            else:
                                pass

                        except NoSuchElementException:
                            pass


                        ### FIND USG 1 ###
                        try:
                            elemFindUSG = browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/preceding-sibling::td[2]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_imagem = [elemRg]
                                string_image = ''
                                elemFindUSG.click()
                                time.sleep(5)

                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)
                                
                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))

                            else:
                                pass
                            
                        except NoSuchElementException:
                            pass

                        ### FIND USG 2 ###
                        try:
                            elemFindUSG_2 = browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/preceding-sibling::td[2]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/..').text.split(' ')[-1] == 'Liberado':
                                list_imagem = [elemRg]
                                string_image = ''
                                
                                elemFindUSG_2.click()
                                time.sleep(5)
                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)

                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            else:
                                pass
                            
                        except NoSuchElementException:
                            pass

                        ### FIND USG 3 ###
                        try:
                            elemFindUSG_3 = browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/preceding-sibling::td[3]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/..').text.split(' ')[-1] == 'Laudado':
                                list_imagem = [elemRg]
                                string_image = ''
                                
                                elemFindUSG_3.click()
                                time.sleep(5)
                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)

                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            else:
                                pass
                        except NoSuchElementException:
                            pass

                        elemSelectExams = WebDriverWait(browser,120).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[1]/table/tbody/tr/td/font/b/u')))
                        elemSelectExams.click()
                        time.sleep(1)

                        try:
                            elemExhibitExames = WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[1]/input')))
                            elemExhibitExames.click()
                            time.sleep(2)

                            WebDriverWait(browser, 60).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="frameRelatorio"]')))

                            elemBody = WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="resultHTML"]')))

                            try:
                                elemBody.send_keys(Keys.COMMAND + 'a')
                                elemBody.send_keys(Keys.COMMAND + 'c')

                                new_string = new_string + '\n' + pyperclip.paste()
                            except ElementNotInteractableException:
                                pass

                            # Voltar
                            elemVoltar = WebDriverWait(browser,60).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnVoltarLista"]')))
                            elemVoltar.click()
                            time.sleep(2)

                            # Limpar
                            browser.switch_to.default_content()
                            WebDriverWait(browser,3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            elemLimpar = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[3]')))
                            elemLimpar.click()
                        
                        except TimeoutException:
                            pass
                        
                        primeiraPagina = browser.find_element(By.XPATH, '/html/body/form/div[3]/div[3]/div[4]/table/tbody/tr[1]/td/table/tbody/tr/td[1]')
                        primeiraPagina.click()
                    
                    except NoSuchElementException:

                        # Only one page found
                        new_string = ''
                        ### FIND ECHO 1 ###
                        try:
                            elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/preceding-sibling::td[2]')
                            time.sleep(3)

                            if browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(5)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')
                                    
                                    string_echo = pyperclip.paste()

                                    ### REGEX ###

                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)

                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                    
                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass
                            
                            elif browser.find_element(By.XPATH, '//td[contains(text(), "ECOCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Laudado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(5)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)

                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_echo = pyperclip.paste()

                                    ### REGEX ###
                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)
                                    
                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)
                                    
                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass
                            else:
                                pass

                        except NoSuchElementException:
                            pass

                        ### FIND ECHO 2 ###
                        try:
                            elemFindEcho = browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/preceding-sibling::td[2]')
                            time.sleep(3)

                            if browser.find_element(By.XPATH, '//td[contains(text(), "ECODOPPLERCARDIOGRAMA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_echo = [elemRg]
                                string_echo = ''
                                elemFindEcho.click()
                                time.sleep(5)

                                try:
                                    WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                    browser.switch_to.window(browser.window_handles[-1])
                                    time.sleep(3)
                                    
                                    elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                    time.sleep(2)
                                    elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                    string_echo = pyperclip.paste()

                                    ### REGEX ###

                                    DATA_EXAME = re.findall(r'Data:\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame = ''.join(DATA_EXAME)
                                    list_echo.append(data_exame[:10])

                                    DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_echo)
                                    data_exame_2 = ''.join(DATA_EXAME_2)
                                    list_echo.append(data_exame_2[:10])

                                    PCA_TAMANHO = re.findall(r'[C-c]anal\s[A-a]rterial\s[P-p]atente\D+(\d+.?\d?)', string_echo)
                                    pca_tamanho = ''.join(PCA_TAMANHO)
                                    list_echo.append(pca_tamanho)

                                    PCA_TAMANHO2 = re.findall(r'[C-c]anal\s[A-a]rterial\D+(\d.?\d?)\s*mm', string_echo)
                                    pca_tamanho2 = ''.join(PCA_TAMANHO2)
                                    list_echo.append(pca_tamanho2)

                                    DIAMETRO_AORTA = re.findall(r'[D-d]i.metro\s[D-d]a\s[A-a]orta\D+(\d+,?\d?)\s*mm', string_echo)
                                    diametro_aorta = ''.join(DIAMETRO_AORTA)
                                    list_echo.append(diametro_aorta)

                                    DIAMETRO_AORTA_2 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_2 = ''.join(DIAMETRO_AORTA_2)
                                    list_echo.append(diametro_aorta_2)

                                    DIAMETRO_AORTA_3 = re.findall(r'[R-r]aiz\s*[A-a].rtica\s*?:\s*?(\d+.?\d?)\s*.m', string_echo)
                                    diametro_aorta_3 = ''.join(DIAMETRO_AORTA_3)
                                    list_echo.append(diametro_aorta_3)

                                    DIAMETRO_AE = re.findall(r'[D-d]i.metro\s[D-d].\s.trio\s[E-e]squerdo\s*(\d+)\s*mm', string_echo)
                                    diametro_ae = ''.join(DIAMETRO_AE)
                                    list_echo.append(diametro_ae)

                                    DIAMETRO_AE_2 = re.findall(r'.trio\s*[E-e]squerdo\s*(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_2 = ''.join(DIAMETRO_AE_2)
                                    list_echo.append(diametro_ae_2)

                                    DIAMETRO_AE_3 = re.findall(r'.trio\s*[E-e]squerdo\s*?:\s*?(\d+.?\d?)\s*cm', string_echo)
                                    diametro_ae_3 = ''.join(DIAMETRO_AE_3)
                                    list_echo.append(diametro_ae_3)

                                    ENCURTAMENTO_VE = re.findall(r'[F-f]ra..o\s*[D-d]e\s*[E-e]ncurtamento\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    encurtamento_ve = ''.join(ENCURTAMENTO_VE)
                                    list_echo.append(encurtamento_ve)

                                    FEVE = re.findall(r'[F-f]ra..o\s[D-d].\s*[E-e]je..o\s*[D-d].\s*VE\s*(\d+)\s*%', string_echo)
                                    feve = ''.join(FEVE)
                                    list_echo.append(feve)

                                    CIV_1 = re.findall(r'[C-c]omunica..o\s[I-i]nterventricular\D+(\d+,?\d?)\s*mm', string_echo)
                                    civ_1 = ''.join(CIV_1)
                                    list_echo.append(civ_1)

                                    VTI_P = re.findall(r'VTI\s[P-p]ulmonar\D+(\d+)\scm', string_echo)
                                    vti_p = ''.join(VTI_P)
                                    list_echo.append(vti_p)

                                    PSAP = re.findall(r'PSAP\D+(\d+)\s*mmHg', string_echo)
                                    psap = ''.join(PSAP)
                                    list_echo.append(psap)
                                
                                    new_df_echo = pd.DataFrame([list_echo])
                                    df_echo = pd.concat([df_echo, new_df_echo], ignore_index=True)

                                    # Close tab
                                    browser.close()
                                    time.sleep(1)
                                    browser.switch_to.window(browser.window_handles[0])
                                    WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                                except TimeoutException:
                                    pass

                            else:
                                pass

                        except NoSuchElementException:
                            pass


                        ### FIND USG 1 ###
                        try:
                            elemFindUSG = browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/preceding-sibling::td[2]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "USG TRANSFONTANELA")]/..').text.split(' ')[-1] == 'Liberado':
                                list_imagem = [elemRg]
                                string_image = ''
                                elemFindUSG.click()
                                time.sleep(5)

                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)
                                
                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))

                            else:
                                pass
                            
                        except NoSuchElementException:
                            pass

                        ### FIND USG 2 ###
                        try:
                            elemFindUSG_2 = browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/preceding-sibling::td[2]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "USG CRANIO")]/..').text.split(' ')[-1] == 'Liberado':
                                list_imagem = [elemRg]
                                string_image = ''
                                
                                elemFindUSG_2.click()
                                time.sleep(5)
                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)

                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            else:
                                pass
                            
                        except NoSuchElementException:
                            pass

                        ### FIND USG 3 ###
                        try:
                            elemFindUSG_3 = browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/preceding-sibling::td[3]')

                            if browser.find_element(By.XPATH, '//td[contains(text(), "CRANIO")]/..').text.split(' ')[-1] == 'Laudado':
                                list_imagem = [elemRg]
                                string_image = ''
                                
                                elemFindUSG_3.click()
                                time.sleep(5)
                                WebDriverWait(browser, 30).until(EC.number_of_windows_to_be(2))

                                browser.switch_to.window(browser.window_handles[-1])
                                time.sleep(3)

                                elemCopyPDF = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, '/html/body')))
                                time.sleep(2)
                                elemCopyPDF.send_keys(Keys.COMMAND + 'a', Keys.COMMAND + 'c')

                                string_image = pyperclip.paste()

                                ### REGEX ###
                                DATA_EXAME = re.findall(r'Data:\s*(\d+\/\d+\/\d+)', string_image)
                                data_exame = ''.join(DATA_EXAME)
                                list_imagem.append(data_exame)

                                DATA_EXAME_2 = re.findall(r'REALIZADO\s*EM\D+(\d+\/\d+\/\d+)', string_image)
                                data_exame_2 = ''.join(DATA_EXAME_2)
                                list_imagem.append(data_exame_2)

                                HEMORRAGIA = re.findall(r'[H-h]emorragia.*[G-g]rau\s*(.*)', string_image)
                                hemorragia = ''.join(HEMORRAGIA)
                                list_imagem.append(hemorragia)

                                new_df_usg = pd.DataFrame([list_imagem])
                                df_usg = pd.concat([df_usg, new_df_usg], ignore_index=True)

                                # Close tab
                                browser.close()
                                time.sleep(1)
                                browser.switch_to.window(browser.window_handles[0])
                                WebDriverWait(browser, 3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            else:
                                pass
                        except NoSuchElementException:
                            pass
                        

                        elemSelectExames = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[1]/table/tbody/tr/td/font/b/u')))
                        elemSelectExames.click()
                        time.sleep(1)

                        try:
                            elemExhibitExames = WebDriverWait(browser,60).until(EC.element_to_be_clickable((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[1]/input')))
                            elemExhibitExames.click()
                            time.sleep(2)

                            WebDriverWait(browser, 60).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="frameRelatorio"]')))
                            elemBody = WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.XPATH, '//*[@id="resultHTML"]')))
                            #elemBody.click()
                            time.sleep(1)

                            try:
                                elemBody.send_keys(Keys.COMMAND + 'a')
                                elemBody.send_keys(Keys.COMMAND + 'c')

                                new_string = new_string + '\n' + pyperclip.paste()
                            except ElementNotInteractableException:
                                pass

                            # Voltar
                            elemVoltar = WebDriverWait(browser,60).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnVoltarLista"]')))
                            elemVoltar.click()
                            time.sleep(2)

                            # Limpar
                            browser.switch_to.default_content()
                            WebDriverWait(browser,3).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, '//*[@id="1"]')))
                            elemLimpar = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[3]/div[3]/div[3]/table/tbody/tr/td[2]/table/tbody/tr/td[3]')))
                            elemLimpar.click()
                            
                        except TimeoutException:
                            pass

                    string = new_string.split("Coletado ")
                    for substring in string:
                        list = [elemRg]

                        DNASC = re.findall(r'NASCIMENTO:\s(\d+\/\d+\/\d+)', substring)
                        dnasc = ''.join(DNASC)
                        list.append(dnasc)

                        Date = re.findall(r'em:\s(\d+.\d+.\d+)', substring)
                        date = ''.join(Date)
                        list.append(date)

                        Hour = re.findall(r'em:\s\d+.\d+.\d+.(\d+:\d+:\d+)', substring)
                        hour = ''.join(Hour)
                        list.append(hour)

                        PH_1 = re.findall(r'pH\s*(\d+,?\d+)\s*7,35', substring)
                        ph_1 = ''.join(PH_1)
                        list.append(ph_1)

                        PO2_1 = re.findall(r'pO2\s*(\d+,?\d+)\smmHg\s*80', substring)
                        po2_1 = ''.join(PO2_1)
                        list.append(po2_1)

                        PCO2_1 = re.findall(r'pCO2\s*(\d+,?\d+)\smmHg\s*3', substring)
                        pco2_1 = ''.join(PCO2_1)
                        list.append(pco2_1)

                        CTCO2_1 = re.findall(r'ctCO2\s*(\d+,?\d+)\smmol\/L\s22', substring)
                        ctco2_1 = ''.join(CTCO2_1)
                        list.append(ctco2_1)

                        CTHCO3_1 = re.findall(r'ctHCO3\s*(\d+,?\d+)\smmol\/L\s21', substring)
                        cthco3_1 = ''.join(CTHCO3_1)
                        list.append(cthco3_1)

                        BE_1 = re.findall(r'BE\s*(.?\d+,?\d+)\smmol\/L\s*-', substring)
                        be_1 = ''.join(BE_1)
                        list.append(be_1)

                        SO2_1 = re.findall(r'SO2\s*(\d+,\d+)\s%', substring)
                        so2_1 = ''.join(SO2_1)
                        list.append(so2_1)

                        FO2Hb_1 = re.findall(r'FO2Hb\s*(\d+,?\d+)\s%\s*95,0', substring)
                        fo2hb_1 = ''.join(FO2Hb_1)
                        list.append(fo2hb_1)

                        FCOHb_1 = re.findall(r'FCOHb\s*(\d+,?\d+)\s%\s*0,5', substring)
                        fcohb_1 = ''.join(FCOHb_1)
                        list.append(fcohb_1)

                        FHHb_1 = re.findall(r'FHHb\s*(\d+,?\d+)\s%\s*1,0', substring)
                        fhhb_1 = ''.join(FHHb_1)
                        list.append(fhhb_1)

                        FMetHb_1 = re.findall(r'FMetHb\s*(\d+,?\d+)\s%\s*0,4', substring)
                        fmethb_1 = ''.join(FMetHb_1)
                        list.append(fmethb_1)

                        GLICOSE = re.findall(r'GLICOSE\s*(\d+)\smg\/dL\s*70', substring)
                        glicose = ''.join(GLICOSE)
                        list.append(glicose)

                        LACTATO = re.findall(r'LACTATO,SANGUE\D+LACTATO\s*(\d+)', substring)
                        lactato = ''.join(LACTATO)
                        list.append(lactato)

                        PHCORDAO = re.findall(r'pH\s*(\d+,?\d+)\s*\(Percentil', substring)
                        phcordao = ''.join(PHCORDAO)
                        list.append(phcordao)

                        HCO3CORDAO = re.findall(r'ctHCO3\s*(\d+,?\d+)\smmol\/L\sArtéria', substring)
                        hco3cordao = ''.join(HCO3CORDAO)
                        list.append(hco3cordao)

                        BECORDAO = re.findall(r'BE\s*(.?\d+,?\d+)\smmol\/L\sArtéria', substring)
                        becordao = ''.join(BECORDAO)
                        list.append(becordao)

                        ERITROCITOS = re.findall(r'Eritrócitos\s*(\d+,?\d+)\smilh', substring)
                        eritrocitos = ''.join(ERITROCITOS)
                        list.append(eritrocitos)

                        HEMOGLOBINA = re.findall(r'Hemoglobina.\s*?(\d+,?\d+)\sg\/dL', substring)
                        hemoglobina = ''.join(HEMOGLOBINA)
                        list.append(hemoglobina)

                        HEMOGLOBINA2 = re.findall(r'HEMOGLOBINA\s*(\d+,?\d+)\sg\/dL', substring)
                        hemoglobina2 = ''.join(HEMOGLOBINA2)
                        list.append(hemoglobina2)

                        HEMATOCRITO = re.findall(r'Hematócrito.\s*?(\d+,?\d+)\s%', substring)
                        hematocrito = ''.join(HEMATOCRITO)
                        list.append(hematocrito)

                        HEMATOCRITO2 = re.findall(r'HEMATOCRITO\s*(\d+,?\d+)\s%', substring)
                        hematocrito2 = ''.join(HEMATOCRITO2)
                        list.append(hematocrito2)

                        VCM = re.findall(r'VCM\s*(\d+,?\d+)\sfL', substring)
                        vcm = ''.join(VCM)
                        list.append(vcm)

                        HCM = re.findall(r'HCM\s*(\d+,?\d+)\spg', substring)
                        hcm = ''.join(HCM)
                        list.append(hcm)

                        CHCM = re.findall(r'CHCM\s*(\d+,?\d+)\sg\/dL', substring)
                        chcm = ''.join(CHCM)
                        list.append(chcm)

                        RDWCV = re.findall(r'RDW-CV\s*(\d+,?\d+)\s%', substring)
                        rdwcv = ''.join(RDWCV)
                        list.append(rdwcv)

                        RDWSD = re.findall(r'RDW-SD\s*(\d+,?\d+)\sfL', substring)
                        rdwsd = ''.join(RDWSD)
                        list.append(rdwsd)

                        ERITROBLASTOS = re.findall(r'Eritroblastos\s*(\d+,?\d+)\s%', substring)
                        eritroblastos = ''.join(ERITROBLASTOS)
                        list.append(eritroblastos)

                        LEUCO = re.findall(r'Leucócitos\s*(\d+,?\d+)\smil\/mm', substring)
                        leuco = ''.join(LEUCO)
                        list.append(leuco)

                        NEUTRO = re.findall(r'Neutrófilos\s(\d+,?\d+)%', substring)
                        neutro = ''.join(NEUTRO)
                        list.append(neutro)

                        PROMIELOPERCENT = re.findall(r'Promielócitos\s*(\d+,?\d+)%', substring)
                        promielopercent = ''.join(PROMIELOPERCENT)
                        list.append(promielopercent)

                        MIELOPERCENT = re.findall(r'Mielócitos\s*(\d+,?\d+)%', substring)
                        mielopercent = ''.join(MIELOPERCENT)
                        list.append(mielopercent)

                        METAMIELOPERCENT = re.findall(r'Metamielócitos\s*(\d+,?\d+)%', substring)
                        metamielopercent = ''.join(METAMIELOPERCENT)
                        list.append(metamielopercent)

                        BASTONETES = re.findall(r'Bastonetes\s*(\d+,?\d+)%', substring)
                        bastonetes = ''.join(BASTONETES)
                        list.append(bastonetes)

                        SEGMENTADOS = re.findall(r'Segmentados\s*(\d+,?\d+)%', substring)
                        segmentados = ''.join(SEGMENTADOS)
                        list.append(segmentados)
                        
                        EOSINO = re.findall(r'Eosinófilos\s(\d+,?\d+)%', substring)
                        eosino = ''.join(EOSINO)
                        list.append(eosino)

                        BASOFILOS = re.findall(r'Basófilos\s(\d+,?\d+)%', substring)
                        basofilos = ''.join(BASOFILOS)
                        list.append(basofilos)

                        LINFO = re.findall(r'Linfócitos\s(\d+,?\d+)%', substring)
                        linfo = ''.join(LINFO)
                        list.append(linfo)

                        MONO = re.findall(r'Monócitos\s(\d+,?\d+)%', substring)
                        mono = ''.join(MONO)
                        list.append(mono)

                        PLAQUETAS = re.findall(r'PLAQUETAS\s*(\d+,?)\smil', substring)
                        plaquetas = ''.join(PLAQUETAS)
                        list.append(plaquetas)

                        VPM = re.findall(r'VPM\s:\s*(\d+,?\d+)\sfL', substring)
                        vpm = ''.join(VPM)
                        list.append(vpm)

                        PCR = re.findall(r'PROTEÍNA\sC\sREATIVA\s\(PCR\)\s*(\d+,?\d+)\smg', substring)
                        pcr = ''.join(PCR)
                        list.append(pcr)

                        SODIO = re.findall(r'SÓDIO\s*(\d+)\smEq\/L\s*135', substring)
                        sodio = ''.join(SODIO)
                        list.append(sodio)

                        POTASSIO = re.findall(r'POTÁSSIO\s*(\d+,?\d+)\smEq\/L', substring)
                        potassio = ''.join(POTASSIO)
                        list.append(potassio)

                        CLORO = re.findall(r'CLORO\s*(\d+,?\d+?)\smEq', substring)
                        cloro = ''.join(CLORO)
                        list.append(cloro)

                        FOSFORO = re.findall(r'FÓSFORO\s*(\d+,?\d+)\smg', substring)
                        fosforo = ''.join(FOSFORO)
                        list.append(fosforo)

                        MAGNESIO = re.findall(r'MAGNÉSIO\s*(\d+,?\d+)\smg\/dL', substring)
                        magnesio = ''.join(MAGNESIO)
                        list.append(magnesio)

                        CATOTAL = re.findall(r'CÁLCIO\sTOTAL\s*(\d+,?\d+)\smg\/dL', substring)
                        catotal = ''.join(CATOTAL)
                        list.append(catotal)

                        CAIONICOmgdl = re.findall(r'CÁLCIO\sIÔNICO\s*(\d+,?\d+)\smg\/dL', substring)
                        caionicomgdl = ''.join(CAIONICOmgdl)
                        list.append(caionicomgdl)

                        UREIA = re.findall(r'URÉIA\s*(\d+,?\d+?)\smg\/dL', substring)
                        ureia = ''.join(UREIA)
                        list.append(ureia)

                        CREAT = re.findall(r'CREATININA\s*(\d+,?\d+)\smg\/dL', substring)
                        creat = ''.join(CREAT)
                        list.append(creat)

                        PROTTOTAL = re.findall(r'PROTEÍNAS\sTOTAIS,SANGUE\D+PROTEÍNAS\sTOTAIS\s*(\d,?\d?)', substring)
                        prottotal = ''.join(PROTTOTAL)
                        list.append(prottotal)

                        ALBUMINA = re.findall(r'ALBUMINA\s*(\d+,?\d+)\sg\/dL', substring)
                        albumina = ''.join(ALBUMINA)
                        list.append(albumina)

                        BT = re.findall(r'BILIRRUBINA\sTOTAL\s*(\d+,?\d+)\smg\/dL', substring)
                        bt = ''.join(BT)
                        list.append(bt)

                        BD = re.findall(r'BILIRRUBINA\sDIRETA\s*(\d+,?\d+)\smg\/dL', substring)
                        bd = ''.join(BD)
                        list.append(bd)

                        BI = re.findall(r'BILIRRUBINA\sINDIRETA\s*(\d+,?\d+)\smg\/dL', substring)
                        bi = ''.join(BI)
                        list.append(bi)

                        TPROTROMBINA = re.findall(r'TEMPO\sDE\sPROTROMBINA\s*(\d+,?\d+)', substring)
                        tprotrombina = ''.join(TPROTROMBINA)
                        list.append(tprotrombina)

                        ATIVPROTROMBINA = re.findall(r'Equivalente\sa\s*(\d+)\s%\sAP', substring)
                        ativprotrombina = ''.join(ATIVPROTROMBINA)
                        list.append(ativprotrombina)

                        INR = re.findall(r'INR\s=\s*(\d+,?\d+)', substring)
                        inr = ''.join(INR)
                        list.append(inr)

                        FIBRINOGENIO = re.findall(r'FIBRINOGÊNIO\s*(\d+)\smg\/dL', substring)
                        fibrinogenio = ''.join(FIBRINOGENIO)
                        list.append(fibrinogenio)

                        TTPA = re.findall(r'TEMPO\sDE\sTROMBOPLASTINA\sPARCIAL\sATIVADA\s*(\d+)\sSegs', substring)
                        ttpa = ''.join(TTPA)
                        list.append(ttpa)

                        RCOAG = re.findall(r'R\s=\s*(\d+,?\d+)', substring)
                        rcoag = ''.join(RCOAG)
                        list.append(rcoag)

                        TTROMBINA = re.findall(r'TEMPO\sDE\sTROMBINA\s*(\d+,?\d+)\sSegs', substring)
                        ttrombina = ''.join(TTROMBINA)
                        list.append(ttrombina)

                        RET = re.findall(r'RETICULÓCITOS\s*(\d+,?\d+)\s%', substring)
                        ret = ''.join(RET)
                        list.append(ret)

                        RETcor = re.findall(r'Contagem\scorrigida\spara\svalor\sde\sHemoglobina\s:\s*(\d+,?\d+)\s%', substring)
                        retcor = ''.join(RETcor)
                        list.append(retcor)

                        RETabs = re.findall(r'Contagem\sabsoluta\scorrigida\s:\s*(\d+)\smil', substring)
                        retabs = ''.join(RETabs)
                        list.append(retabs)

                        ERIGLOBAL = re.findall(r'Valor\sglobal\sde\sEritrocitos\s:\s*(\d+,?\d+)\smi', substring)
                        eriglobal = ''.join(ERIGLOBAL)
                        list.append(eriglobal)

                        IRF = re.findall(r'\(IRF\s%\)\s:\s*(\d+,?\d+)\s%', substring)
                        irf = ''.join(IRF)
                        list.append(irf)

                        RETHE = re.findall(r'Ret-He\s:\s*(\d+,?\d+)\spg', substring)
                        rethe = ''.join(RETHE)
                        list.append(rethe)

                        FERRITINA = re.findall(r'FERRITINA\s*(\d+)\sng', substring)
                        ferritina = ''.join(FERRITINA)
                        list.append(ferritina)

                        TIPAGEM = re.findall(r'TIPAGEM\sSANGUÍNEA\sABO-Rh\(D\)\s*(.+)Hemaglutina', substring)
                        tipagem = ''.join(TIPAGEM)
                        list.append(tipagem)

                        COOMBSDIRETO = re.findall(r'TESTE\D+TADC\s*(\w+)', substring)
                        coombsdireto = ''.join(COOMBSDIRETO)
                        list.append(coombsdireto)

                        TSH = re.findall(r'HORMÔNIO\sTIREO-ESTIMULANTE\s\(TSH\)\s*(\d+,?\d+)\s', substring)
                        tsh = ''.join(TSH)
                        list.append(tsh)

                        T4L = re.findall(r'TIROXINA\sLIVRE\s\(T4L\)\s*(\d+,?\d+)\sng', substring)
                        t4l = ''.join(T4L)
                        list.append(t4l)

                        T3 = re.findall(r'TRIIODOTIRONINA\s\(T3\)\s*(\d+)', substring)
                        t3 = ''.join(T3)
                        list.append(t3)

                        T4 = re.findall(r'TIROXINA\s\(T4\)\s*(\d+,?\d+)', substring)
                        t4 = ''.join(T4)
                        list.append(t4)

                        VITD = re.findall(r'25-HIDROXIVITAMINA\sD\s*(\d+,?\d+)', substring)
                        vitd = ''.join(VITD)
                        list.append(vitd)

                        LCRcels = re.findall(r'Celulas\s:\s*(\d+)\s\/mm.\sAté\s4\scélulas\s\/mm', substring)
                        lcrcels = ''.join(LCRcels)
                        list.append(lcrcels)

                        LCRhem = re.findall(r'Hemacias\s:\s*(\d+)\s\/mm.\sAté\s:\s0', substring)
                        lcrhem = ''.join(LCRhem)
                        list.append(lcrhem)

                        LCRglic = re.findall(r'GLICOSE,LÍQUOR\D+GLICOSE\s*(\d+)\smg\/dL', substring)
                        lcrglic = ''.join(LCRglic)
                        list.append(lcrglic)

                        LCRlact = re.findall(r'LACTATO,LÍQUOR\D+LACTATO\s*(\d+,?\d?)', substring)
                        lcrlact = ''.join(LCRlact)
                        list.append(lcrlact)

                        LCRprot = re.findall(r'PROTEÍNAS\sTOTAIS,LÍQUOR\D+PROTEÍNAS\sTOTAIS\s*(\d+)', substring)
                        lcrprot = ''.join(LCRprot)
                        list.append(lcrprot)

                        HMCPERI = re.findall(r'CULTURA\sAERÓBIA,SANGUE\sPERIFÉRICO\s.*\n\nCULTURA\sAERÓBIA\s*(\w+\s?\w+)', substring)
                        hmcperi = ''.join(HMCPERI)
                        list.append(hmcperi)

                        UROC = re.findall(r'CULTURA\sAERÓBIA,URINA\sDE\sSONDA\sVESICAL\sDE\D+CULTURA\sAERÓBIA\s*(\w+\s\w+)', substring)
                        uroc = ''.join(UROC)
                        list.append(uroc)

                        FUNGOLCR = re.findall(r'CULTURA\sPARA\sFUNGOS,LÍQUOR\D+CULTURA\sPARA\sFUNGOS\s*(\w+\s?\w+?)', substring)
                        fungolcr = ''.join(FUNGOLCR)
                        list.append(fungolcr)

                        FUNGOURINA = re.findall(r'CULTURA\sPARA\sFUNGOS,URINA\sDE\sSONDA\sVESICAL\sDE\sALÍVIO\D+CULTURA\sPARA\sFUNGOS\s*(\w+\s?\w+)', substring)
                        fungourina = ''.join(FUNGOURINA)
                        list.append(fungourina)

                        CULTLCR = re.findall(r'CULTURA\sAERÓBIA,LÍQUOR\s.*\n\nCULTURA\sAERÓBIA\s*(.*)\s*Negativa', substring)
                        cultlcr = ''.join(CULTLCR)
                        list.append(cultlcr)

                        CARIOTIPO = re.findall(r'CARIOTIPO...BANDA\sG,SANGUE\s.*\n\nResultado\s:\s*(.*)Feminino', substring)
                        cariotipo = ''.join(CARIOTIPO)
                        list.append(cariotipo)

                        PCRCOVID = re.findall(r'Coronavírus\s2019-nCoV\s*(.*)\sNão\sdetectado', substring)
                        pcrcovid = ''.join(PCRCOVID)
                        list.append(pcrcovid)

                        NSFENO = re.findall(r'FENOBARBITAL\s*(\d+,?\d+)\s.g\/mL', substring)
                        nsfeno = ''.join(NSFENO)
                        list.append(nsfeno)

                        NSVANCO = re.findall(r'VANCOMICINA\s*(\d+,?\d+)', substring)
                        nsvanco = ''.join(NSVANCO)
                        list.append(nsvanco)

                        U1PH = re.findall(r'URINA\sTIPO\s1\D+pH\D+(\d+,?\d+?)', substring)
                        u1ph = ''.join(U1PH)
                        list.append(u1ph)

                        U1DENSIDADE = re.findall(r'Densidade\s:\s*(\d+,?\d+)', substring)
                        u1densidade = ''.join(U1DENSIDADE)
                        list.append(u1densidade)

                        U1GLICOSE  = re.findall(r'Densidade.*\nGlicose\s:\s*(.*)\sAusente', substring)
                        u1glicose = ''.join(U1GLICOSE)
                        list.append(u1glicose)

                        U1PIGBILIARES = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares\s:\s*(.*)\sAusente', substring)
                        u1pigbilirares = ''.join(U1PIGBILIARES)
                        list.append(u1pigbilirares)

                        U1CORPOSCETONICOS = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares.*\nCorpos\sCetonicos\s:\s*(.*)\sAusente', substring)
                        u1corposcetonicos = ''.join(U1CORPOSCETONICOS)
                        list.append(u1corposcetonicos)

                        U1SANGUE = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares.*\nCorpos\sCetonicos.*\nSangue\s:\s*(.*)\sAusente', substring)
                        u1sangue = ''.join(U1SANGUE)
                        list.append(u1sangue)

                        U1PROT = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares.*\nCorpos\sCetonicos.*\nSangue.*\nProteinas\s:\s*(\d+,\d+)', substring)
                        u1prot = ''.join(U1PROT)
                        list.append(u1prot)

                        U1UROBILINOG = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares.*\nCorpos\sCetonicos.*\nSangue.*\nProteinas.*\nUrobilinogenio\s:\s*(\d+,?\d+)', substring)
                        u1urobilinog = ''.join(U1UROBILINOG)
                        list.append(u1urobilinog)

                        U1NITRITO = re.findall(r'Densidade.*\n.*\nPigmento\sBiliares.*\nCorpos\sCetonicos.*\nSangue.*\nProteinas.*\nUrobilinogenio.*\nNitrito.\s*(.*)Negativo', substring)
                        u1nitrito = ''.join(U1NITRITO)
                        list.append(u1nitrito)

                        U1CELSEPITELIAIS = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais..\s*(.*)Raras', substring)
                        u1celsepiteliais = ''.join(U1CELSEPITELIAIS)
                        list.append(u1celsepiteliais)

                        U1FILAMENTOSDEMUCO = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco..\s*(.*)Ausente', substring)
                        u1filamentosdemuco = ''.join(U1FILAMENTOSDEMUCO)
                        list.append(u1filamentosdemuco)

                        U1LEUCOPORCAMPO = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco.*\nLeucocitos..\s*(.*)\/campo', substring)
                        u1leucoporcampo = ''.join(U1LEUCOPORCAMPO)
                        list.append(u1leucoporcampo)

                        U1ERITROPORCAMPO = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco.*\nLeucocitos.*\nEritrócitos..\s*(\d+)', substring)
                        u1eritroporcampo = ''.join(U1ERITROPORCAMPO)
                        list.append(u1eritroporcampo)

                        U1MORFOLOGIA = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco.*\nLeucocitos.*\nEritrócitos.*\nMorfologia..\s*(.*)', substring)
                        u1morfologia = ''.join(U1MORFOLOGIA)
                        list.append(u1morfologia)

                        U1CILINDRO = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco.*\nLeucocitos.*\nEritrócitos.*\nMorfologia.*\nCilindro..\s*(.*)Ausentes', substring)
                        u1cilindro = ''.join(U1CILINDRO)
                        list.append(u1cilindro)

                        U1CRISTAIS = re.findall(r'SEDIMENTO.\(aumento.400x\).*\nCelulas\sEpiteliais.*\nFilamentos\sde\sMuco.*\nLeucocitos.*\nEritrócitos.*\nMorfologia.*\nCilindro.*\nCristais..\s*(.*)Ausentes', substring)
                        u1cristais = ''.join(U1CRISTAIS)
                        list.append(u1cristais)

                        FOSFATASEALC = re.findall(r'FOSFATASE\sALCALINA\D+37.C\):\s*(\d+)\sU\/L', substring)
                        fosfatasealc = ''.join(FOSFATASEALC)
                        list.append(fosfatasealc)

                        TGP = re.findall(r'ALANINA\sAMINOTRANSFERASE\s*(\d+)\sU\/L', substring)
                        tgp = ''.join(TGP)
                        list.append(tgp)

                        TGO = re.findall(r'ASPARTATO\sAMINO\sTRANSFERASE\s*(\d+)\sU\/L', substring)
                        tgo = ''.join(TGO)
                        list.append(tgo)

                        CAI_2MMOL = re.findall(r'CALCIO\sIONICO\D+(\d+,?\d+?)\smmol', substring)
                        cai_2mmol = ''.join(CAI_2MMOL)
                        list.append(cai_2mmol)

                        CAT_2 = re.findall(r'CALCIO\s\(SORO\)\D+(\d+,?\d+)\smg\/dL', substring)
                        cat_2 = ''.join(CAT_2)
                        list.append(cat_2)

                        HMC_CATETER = re.findall(r'CULTURA\sAERÓBIA,SANGUE\s\(CATETER\)\s.*\n\nCULTURA\sAERÓBIA\s*(\w+\s?\w+)', substring)
                        hmc_cateter = ''.join(HMC_CATETER)
                        list.append(hmc_cateter)

                        HMC_CATETER_2 = re.findall(r'CULTURA\sAERÓBIA,CATÉTER\s.*\n\nCULTURA\sAERÓBIA\s*(\w+\s?\w+)', substring)
                        hmc_cateter_2 = ''.join(HMC_CATETER_2)
                        list.append(hmc_cateter_2)

                        FOSFATASEALC_2 = re.findall(r'FOSFATASE\sALCALINA\s*(\d+)\sU\/L', substring)
                        fosfatasealc_2 = ''.join(FOSFATASEALC_2)
                        list.append(fosfatasealc_2)

                        GGT = re.findall(r'GAMA\sGLUTAMIL\sTRANSFERASE\s*(\d+)\sU\/L', substring)
                        ggt = ''.join(GGT)
                        list.append(ggt)

                        TG = re.findall(r'TRIGLICÉRIDES\s*(\d+)\smg\/dL', substring)
                        tg = ''.join(TG)
                        list.append(tg)

                        ALFA1_ANTITRIP = re.findall(r'ALFA\s1\sANTITRIPSINA\s*(\d+)\smg\/dL', substring)
                        alfa1_antitrip = ''.join(ALFA1_ANTITRIP)
                        list.append(alfa1_antitrip)

                        ADALCR = re.findall(r'ADENOSINA\sDEAMINASE,LÍQUOR\D+(\d+,?\d+?)\sU', substring)
                        adalcr = ''.join(ADALCR)
                        list.append(adalcr)

                        SWABRETAL = re.findall(r'CULTURA\sAERÓBIA,SWAB\sRETAL\D+CULTURA\sAERÓBIA\s*(\w+\s?\w+)', substring)
                        swabretal = ''.join(SWABRETAL)
                        list.append(swabretal)

                        ERCRETAL = re.findall(r'PESQUISA\sDE\sENTEROBACTÉRIAS\sCARBAPENEM\sRESISTENTES\s*(\w+\s?\w+)', substring)
                        ercretal = ''.join(ERCRETAL)
                        list.append(ercretal)

                        ENTEROVANCOR = re.findall(r'PESQUISA\sDE\sENTEROCOCCUS\sVANCOMICINA\sRESISTENTE\s*(\w+\s?\w+)', substring)
                        enterovancor = ''.join(ENTEROVANCOR)
                        list.append(enterovancor)

                        PESQFUNGOLCR = re.findall(r'PESQUISA\sDE\sFUNGOS,LÍQUOR\D+PESQUISA\sDE\sFUNGOS\s*(\w+\s?\w+)', substring)
                        pesqfungolcr = ''.join(PESQFUNGOLCR)
                        list.append(pesqfungolcr)

                        UROCSACOCOLETOR = re.findall(r'CULTURA\sAERÓBIA,URINA\sDE\sCOLETOR\sPLÁSTICO\D+CULTURA\sAERÓBIA\s*(\w+\s?\w+)', substring)
                        urocsacocoletor = ''.join(UROCSACOCOLETOR)
                        list.append(urocsacocoletor)

                        HBA1 = re.findall(r'Hemoglobina\sA1..\s*(\d+,?\d+?)\s%', substring)
                        hba1 = ''.join(HBA1)
                        list.append(hba1)

                        HBA2 = re.findall(r'Hemoglobina\sA2..\s*(\d+,?\d+?)\s%', substring)
                        hba2 = ''.join(HBA2)
                        list.append(hba2)

                        HBF = re.findall(r'Hemoglobina\sFetal..\s*(\d+,?\d+?)\s%', substring)
                        hbf = ''.join(HBF)
                        list.append(hbf)

                        pH_V = re.findall(r'pH\s*(\d,?\d+)\s*7,32\s-', substring)
                        ph_v = ''.join(pH_V)
                        list.append(ph_v)

                        pCO2_V = re.findall(r'pCO2\s*(\d+,?\d+?)\smmHg\s*38\s-', substring)
                        pco2_v = ''.join(pCO2_V)
                        list.append(pco2_v)

                        BIC_V = re.findall(r'HCO3\s*(\d+,?\d+?)\smmol\/L\s*22\s-', substring)
                        bic_v = ''.join(BIC_V)
                        list.append(bic_v)

                        BE_V = re.findall(r'BE\s*(-?\d+,?\d+)\smmol\/L\s*-3', substring)
                        be_v = ''.join(BE_V)
                        list.append(be_v)

                        LACTATO_V = re.findall(r'LACTATO\s,\sSANGUE\sTOTAL\n\nRESULTADO:\s(\d+,\d+)\smg\/dL\s*4,5', substring)
                        lactato_v = ''.join(LACTATO_V)
                        list.append(lactato_v)

                        GLICOSE_V = re.findall(r'GLICOSE..SANGUE\sTOTAL\n\nRESULTADO:\s*(\d+)\smg\/dL\s*50', substring)
                        glicose_v = ''.join(GLICOSE_V)
                        list.append(glicose_v)

                        CAI_V = re.findall(r'CALCIO\sIONICO,\sSANGUE\sTOTAL\n\nRESULTADO:\s*(\d+,?\d+)\smmol\/L\s*1,22', substring)
                        cai_v = ''.join(CAI_V)
                        list.append(cai_v)

                        PCR_2 = re.findall(r'PROTEINA\sC\sREATIVA\s\(SORO\)\D+(\d+,\d+)\smg\/L\s*<5,0', substring)
                        pcr_2 = ''.join(PCR_2)
                        list.append(pcr_2)

                        PCRADENOVIRUS = re.findall(r'Adenovírus\shumano\s\(HAdV\)\s*(.*)PCR\sem\sTempo\sReal', substring)
                        pcradenovirus = ''.join(PCRADENOVIRUS)
                        list.append(pcradenovirus)

                        HADVLCR = re.findall(r'LÍQUOR\n\n\D+\(HAdV\)\s*(.*)Não\s.etectado', substring)
                        hadvlcr = ''.join(HADVLCR)
                        list.append(hadvlcr)

                        CMVLCR = re.findall(r'LÍQUOR\n\n\D+\(CMV\)\s*(.*)Não\s.etectado', substring)
                        cmvlcr = ''.join(CMVLCR)
                        list.append(cmvlcr)

                        EBVLCR = re.findall(r'LÍQUOR\n\n\D+\(EBV\)\s*(.*)Não\s.etectado', substring)
                        ebvlcr = ''.join(EBVLCR)
                        list.append(ebvlcr)

                        EVLCR = re.findall(r'LÍQUOR\n\n\D+\(EV\)\s*(.*)Não\s.etectado', substring)
                        evlcr = ''.join(EVLCR)
                        list.append(evlcr)

                        PARVOLCR = re.findall(r'LÍQUOR\n\n.*\(PARVOVÍRUS\sB19\)\s*(.*)Não\s.etectado', substring)
                        parvolcr = ''.join(PARVOLCR)
                        list.append(parvolcr)

                        HERPESLCR = re.findall(r'LÍQUOR\n\n.*HERPES\sSIMPLEX\sI\/II\s*(.*)Não\s.etectado', substring)
                        herpeslcr = ''.join(HERPESLCR)
                        list.append(herpeslcr)

                        HHV6e7LCR = re.findall(r'LÍQUOR\n\n.*\(HHV6\se\s7\)\s*(.*)Não\s.etectado', substring)
                        hhv6e7lcr = ''.join(HHV6e7LCR)
                        list.append(hhv6e7lcr)

                        HPEVLCR = re.findall(r'LÍQUOR\n\n.*\(HPeV\)\s*(.*)Não\s.etectado', substring)
                        hpevlcr = ''.join(HPEVLCR)
                        list.append(hpevlcr)

                        VZVLCR = re.findall(r'LÍQUOR\n\n.*\(VZV\)\s*(.*)Não\s.etectado', substring)
                        vzvlcr = ''.join(VZVLCR)
                        list.append(vzvlcr)

                        VASADENOPCR = re.findall(r'Adenovírus\shumano\s\(HAdV\),SWAB\D+HAdV\)\s*(.*)Não\s.etectado', substring)
                        vasadenopcr = ''.join(VASADENOPCR)
                        list.append(vasadenopcr)

                        SARSCOVSWAB = re.findall(r'SARS-CoV-2\s*(.*)Não\s.etectado', substring)
                        sarscovswab = ''.join(SARSCOVSWAB)
                        list.append(sarscovswab)

                        VASREV = re.findall(r'Rinovírus\D+\(REV\)\s*(.*)Não\s.etectado', substring)
                        vasrev = ''.join(VASREV)
                        list.append(vasrev)

                        VASMETAPNEUMO = re.findall(r'Metapneumovírus\shumano\s*(.*)Não\s.etectado', substring)
                        vasmetapneumo = ''.join(VASMETAPNEUMO)
                        list.append(vasmetapneumo)

                        VASMYCOPLASMA = re.findall(r'Mycoplasma\spneumoniae\s\(Mpneu\)\s*(.*)Não\s.etectado', substring)
                        vasmycoplasma = ''.join(VASMYCOPLASMA)
                        list.append(vasmycoplasma)

                        VASINFLUENZA_A = re.findall(r'Influenza\sA\s\/\sH1N12009\s\/\sH3N2\s*(.*)Não\s.etectado', substring)
                        vasinfluenza_a = ''.join(VASINFLUENZA_A)
                        list.append(vasinfluenza_a)

                        VASINFLUENZA_B = re.findall(r'Influenza\sB\s\(FLU\sB\)\s*(.*)Não\sdetectado', substring)
                        vasinfluenza_b = ''.join(VASINFLUENZA_B)
                        list.append(vasinfluenza_b)

                        VASPARA1 = re.findall(r'\(HPIV1\)\s*(.*)Não\s.etectado', substring)
                        vaspara1 = ''.join(VASPARA1)
                        list.append(vaspara1)

                        VASPARA2 = re.findall(r'\(HPIV2\)\s*(.*)Não\s.etectado', substring)
                        vaspara2 = ''.join(VASPARA2)
                        list.append(vaspara2)

                        VASPARA3 = re.findall(r'\(HPIV3\)\s*(.*)Não\s.etectado', substring)
                        vaspara3 = ''.join(VASPARA3)
                        list.append(vaspara3)

                        VASPARA4 = re.findall(r'\(HPIV4\)\s*(.*)Não\s.etectado', substring)
                        vaspara4 = ''.join(VASPARA4)
                        list.append(vaspara4)

                        VASVSR = re.findall(r'SWAB\D+VÍRUS\sSINCICIAL\sRESPIRATÓRIO\s*(.*)Não\s.etectado', substring)
                        vasvsr = ''.join(VASVSR)
                        list.append(vasvsr)

                        VASCHLAMYDIA = re.findall(r'SWAB\D+Chlamydia\spneumoniae\s*(.*)Não\s.etectado', substring)
                        vaschlamydia = ''.join(VASCHLAMYDIA)
                        list.append(vaschlamydia)

                        RESCLINDA = re.findall(r'CLINDAMICINA.-.(.*)', substring)
                        resclinda = ''.join(RESCLINDA).rstrip()
                        list.append(resclinda)

                        RESOXA = re.findall(r'OXACILINA....(.*)', substring)
                        resoxa = ''.join(RESOXA).rstrip()
                        list.append(resoxa)

                        RESERITRO = re.findall(r'ERITROMICINA...(.*)', substring)
                        reseritro = ''.join(RESERITRO).rstrip()
                        list.append(reseritro)

                        RESGENTA = re.findall(r'GENTAMICINA...(.*)', substring)
                        resgenta = ''.join(RESGENTA).rstrip()
                        list.append(resgenta)

                        RESLEVO = re.findall(r'LEVOFLOXACINA...(.*)', substring)
                        reslevo = ''.join(RESLEVO).rstrip()
                        list.append(reslevo)

                        RESLINEZ = re.findall(r'LINEZOLIDA...(.*)',substring)
                        reslinez = ''.join(RESLINEZ).rstrip()
                        list.append(reslinez)

                        RESPENI = re.findall(r'PENICILINA...(.*)', substring)
                        respeni = ''.join(RESPENI).rstrip()
                        list.append(respeni)

                        RESRIFAMP = re.findall(r'RIFAMPICINA...(.*)', substring)
                        resrifamp = ''.join(RESRIFAMP).rstrip()
                        list.append(resrifamp)

                        RESSULFA = re.findall(r'SULFA\s\+\sTRIMETHOPRIM...(.*)', substring)
                        ressulfa = ''.join(RESSULFA).rstrip()
                        list.append(ressulfa)

                        RESTEICO = re.findall(r'TEICOPLANINA...(.*)', substring)
                        resteico = ''.join(RESTEICO).rstrip()
                        list.append(resteico)

                        RESTIGECI = re.findall(r'TIGECICLINA...(.*)', substring)
                        restigeci = ''.join(RESTIGECI).rstrip()
                        list.append(restigeci)

                        RESVANCO = re.findall(r'VANCOMICINA...(.*)]', substring)
                        resvanco = ''.join(RESVANCO).rstrip()
                        list.append(resvanco)

                        RESERTAP = re.findall(r'ERTAPENEM...(.*)', substring)
                        resertap = ''.join(RESERTAP).rstrip()
                        list.append(resertap)

                        ESBL = re.findall(r'ESBL...(.*)', substring)
                        esbl = ''.join(ESBL).rstrip()
                        list.append(esbl)

                        RESNITROF = re.findall(r'NITROFURANTOINA...(.*)', substring)
                        resnitrof = ''.join(RESNITROF).rstrip()
                        list.append(resnitrof)

                        RESAMICA = re.findall(r'AMICACINA...(.*)', substring)
                        resamica = ''.join(RESAMICA).rstrip()
                        list.append(resamica)

                        RESAMPI = re.findall(r'AMPICILINA...(.*)', substring)
                        resampi = ''.join(RESAMPI).rstrip()
                        list.append(resampi)

                        RESCEFEP = re.findall(r'CEFEPIME...(.*)', substring)
                        rescefep = ''.join(RESCEFEP).rstrip()
                        list.append(rescefep)

                        RESCEFOX = re.findall(r'CEFOXITINA...(.*)', substring)
                        rescefox = ''.join(RESCEFOX).rstrip()
                        list.append(rescefox)

                        RESCEFTAZ = re.findall(r'CEFTAZIDIMA...(.*)', substring)
                        resceftaz = ''.join(RESCEFTAZ).rstrip()
                        list.append(resceftaz)

                        RESCEFTRIAX = re.findall(r'CEFTRIAXONE...(.*)', substring)
                        resceftriax = ''.join(RESCEFTRIAX).rstrip()
                        list.append(resceftriax)

                        RESCEFUROX = re.findall(r'CEFUROXIMA...(.*)', substring)
                        rescefurox = ''.join(RESCEFUROX).rstrip()
                        list.append(rescefurox)

                        RESCIPRO = re.findall(r'CIPROFLOXACINA...(.*)', substring)
                        rescipro = ''.join(RESCIPRO).rstrip()
                        list.append(rescipro)

                        RESIMIPENEM = re.findall(r'IMIPENEM...(.*)', substring)
                        resimipenem = ''.join(RESIMIPENEM).rstrip()
                        list.append(resimipenem)

                        RESMERO = re.findall(r'MEROPENEM...(.*)', substring)
                        resmero = ''.join(RESMERO).rstrip()
                        list.append(resmero)

                        RESPIPETAZO = re.findall(r'PIPERACILINA\/TAZOBACTAM...(.*)', substring)
                        respipetazo = ''.join(RESPIPETAZO).rstrip()
                        list.append(respipetazo)

                        COV2_T = re.findall(r'CoV2-T.*\n.*\n\nIndice:\s*(\d+,\d+)', substring)
                        cov2_t = ''.join(COV2_T)
                        list.append(cov2_t)

                        HBS = re.findall(r'Hemoglobina\sS:\s*(\d+,\d+)', substring)
                        hbs = ''.join(HBS)
                        list.append(hbs)

                        VDRLSANGUE = re.findall(r'VDRL,SANGUE\n.*\n\nVDRL\s*(.*)Não\s.eagente', substring)
                        vdrlsangue = ''.join(VDRLSANGUE).rstrip()
                        list.append(vdrlsangue)

                        VDRLSANGUETITULO = re.findall(r'VDRL,SANGUE\n.*\n\nVDRL.*\nTítulo:\s*(.*)', substring)
                        vdrlsanguetitulo = ''.join(VDRLSANGUETITULO).rstrip()
                        list.append(vdrlsanguetitulo)

                        VDRLLCR = re.findall(r'VDRL,LÍQUOR\n.*\n\nVDRL\s*(.*)Não\s.eagente', substring)
                        vdrllcr = ''.join(VDRLLCR).rstrip()
                        list.append(vdrllcr)

                        PCRTOXOLCR = re.findall(r'LÍQUOR\D+TOXOPLASMA\sGONDII\s\(TOXO\)\sPOR\sPCR\s*(.*)Não\s.etectado', substring)
                        pcrtoxolcr = ''.join(PCRTOXOLCR).rstrip()
                        list.append(pcrtoxolcr)

                        TOXOIGG = re.findall(r'TOXOPLASMOSE...IgG\s*(.*)Não\s.eagente', substring)
                        toxoigg = ''.join(TOXOIGG).rstrip()
                        list.append(toxoigg)

                        TOXOIGM = re.findall(r'TOXOPLASMOSE...IgM\s*(.*)Não\s.eagente', substring)
                        toxoigm = ''.join(TOXOIGM).rstrip()
                        list.append(toxoigm)

                        ANTIXA = re.findall(r'Anti\sFator\sX\sativado:\s*(\d+,\d+)', substring)
                        antixa = ''.join(ANTIXA)
                        list.append(antixa)

                        CKMB = re.findall(r'CK-MB\D+(\d+,\d+)\sng\/mL', substring)
                        ckmb = ''.join(CKMB)
                        list.append(ckmb)

                        CK = re.findall(r'CREATINO\sFOSFOQUINASE\s*(\d+)\sU\/L', substring)
                        ck = ''.join(CK)
                        list.append(ck)

                        DDIMERO = re.findall(r'DOSAGEM\sDO\sDÍMERO-D\sQUANTITATIVO\s*(\d+)\sng\/mL', substring)
                        ddimero = ''.join(DDIMERO)
                        list.append(ddimero)

                        HBC = re.findall(r'Hemoglobina\sC..\s*(\d+,\d+)', substring)
                        hbc = ''.join(HBC)
                        list.append(hbc)

                        TROPONINA = re.findall(r'TROPONINA\sT\s*(\d+,\d+)', substring)
                        troponina = ''.join(TROPONINA)
                        list.append(troponina)

                        CORTISOL = re.findall(r'CORTISOL\s*(\d+,\d+)', substring)
                        cortisol = ''.join(CORTISOL)
                        list.append(cortisol)

                        ANTIRO = re.findall(r'ANTI-SS-A\/RO\s*(\w+)\s*<.7', substring)
                        antiro = ''.join(ANTIRO)
                        list.append(antiro)

                        ANTILA = re.findall(r'ANTI-SS-B\/LA\s*(\w+)\s*<.7', substring)
                        antila = ''.join(ANTILA)
                        list.append(antila)

                        CMVURINA = re.findall(r'DETECÇÃO\sQUANTITATIVA\sDE\sDNA\sDE\sCITOMEGALOVIRUS\s\(CMV\)\D+(\d+)\sUI\/mL', substring)
                        cmvurina = ''.join(CMVURINA)
                        list.append(cmvurina)

                        ANTITG = re.findall(r'ANTICORPO\sANTI-TIREOGLOBULINA\s*(.*)<\s4\sIU\/mL', substring)
                        antitg = ''.join(ANTITG).rstrip()
                        list.append(antitg)

                        ANTITPO = re.findall(r'ANTICORPO\sANTI-PEROXIDASE\s\(ANTI-TPO\)\s*(.*)<\s9\sIU\/mL', substring)
                        antitpo = ''.join(ANTITPO).rstrip()
                        list.append(antitpo)

                        HCO3BIC = re.findall(r'HCO3\s*(\d+,?\d+)\smmol\/L\s*21', substring)
                        hco3bic = ''.join(HCO3BIC)
                        list.append(hco3bic)

                        SATO2GASO = re.findall(r'Saturação\sO2\s*(\d+,?\d+)\s%\s*95', substring)
                        sato2gaso = ''.join(SATO2GASO)
                        list.append(sato2gaso)

                        GLICGASO = re.findall(r'GLICOSE,\sSANGUE\sTOTAL\sARTERIAL\D+(\d+)\smg\/dL', substring)
                        glicgaso = ''.join(GLICGASO)
                        list.append(glicgaso)

                        LACTATOGASO = re.findall(r'LACTATO,\sSANGUE\sTOTAL\sARTERIAL\D+(\d+,?\d+)\smg\/dL', substring)
                        lactatogaso = ''.join(LACTATOGASO)
                        list.append(lactatogaso)

                        RDW_2 = re.findall(r'RDW\s(\d+,\d+)\s%\s*11,60', substring)
                        rdw_2 = ''.join(RDW_2)
                        list.append(rdw_2)

                        LEUCO_ABSOLUTO = re.findall(r'LEUCÓCITOS:\s(\d+,\d+)\smil\/mm3', substring)
                        leuco_absoluto = ''.join(LEUCO_ABSOLUTO)
                        list.append(leuco_absoluto)

                        MIELOB_ABSOLUTO = re.findall(r'Mieloblasto:\s(\d+,\d+)\s', substring)
                        mielob_absoluto = ''.join(MIELOB_ABSOLUTO)
                        list.append(mielob_absoluto)

                        PROMIELO_ABSOLUTO = re.findall(r'Promielócito\sneutrofilo:\s*(\d+,?\d+)\s', substring)
                        promielo_absoluto = ''.join(PROMIELO_ABSOLUTO)
                        list.append(promielo_absoluto)

                        MIELO_ABSOLUTO = re.findall(r'Mielócito\sneutrófilo:\s*(\d+,?\d+)\s', substring)
                        mielo_absoluto = ''.join(MIELO_ABSOLUTO)
                        list.append(mielo_absoluto)

                        METAMIELO_ABSOLUTO = re.findall(r'Metamielócito\sneutrófilo:\s*(\d+,?\d+)\s', substring)
                        metamielo_absoluto = ''.join(METAMIELO_ABSOLUTO)
                        list.append(metamielo_absoluto)

                        BAST_ABSOLUTO = re.findall(r'Bastonete\sneutrófilo:\s*(\d+,?\d+)\s', substring)
                        bast_absoluto = ''.join(BAST_ABSOLUTO)
                        list.append(bast_absoluto)

                        SEG_ABSOLUTO = re.findall(r'Segmentado\sneutrófilo:\s*(\d+,?\d+)\s', substring)
                        seg_absoluto = ''.join(SEG_ABSOLUTO)
                        list.append(seg_absoluto)

                        LINF_ABSOLUTO = re.findall(r'Linfócito:\s*(\d+,?\d+)\s', substring)
                        linf_absoluto = ''.join(LINF_ABSOLUTO)
                        list.append(linf_absoluto)

                        MONO_ABSOLUTO = re.findall(r'Monócito:\s*(\d+,?\d+)\s', substring)
                        mono_absoluto = ''.join(MONO_ABSOLUTO)
                        list.append(mono_absoluto)

                        EOSINO_ABSOLUTO = re.findall(r'Eosinófilo:\s*(\d+,?\d+)\s', substring)
                        eosino_absoluto = ''.join(EOSINO_ABSOLUTO)
                        list.append(eosino_absoluto)

                        BASO_ABSOLUTO = re.findall(r'Basófilo:\s*(\d+,?\d+)\s', substring)
                        baso_absoluto = ''.join(BASO_ABSOLUTO)
                        list.append(baso_absoluto)

                        BLASTOS = re.findall(r'Blastos:\s*(\d+,?\d+)\s', substring)
                        blastos = ''.join(BLASTOS)
                        list.append(blastos)

                        PLAQUETAS_2 = re.findall(r'PLAQUETAS\n\nRESULTADO:\s*(\d+)\sK', substring)
                        plaquetas_2 = ''.join(PLAQUETAS_2)
                        list.append(plaquetas_2)

                        list.append(nome_paciente)

                        df = df.append([list], ignore_index=True)

                    df = df.rename(columns={0:'RGHC', 1:'DNASC', 2:'Date', 3:'Hour', 4:'pH', 5:'pO2', 6:'pCO2', 7:'ctCO2', 8:'BIC', 9:'BE', 10:'SO2', 11:'FO2Hb', 12:'FCOHb', 13:'FHHb', 14:'FMetHb', 15:'Glic', 16:'Lactato', 17:'pHcordao', 18:'BICcordao', 19: 'BEcordao', 20:'Eritrócitos', 21:'Hb', 22:'Hb2', 23:'Ht', 24:'Ht2', 25:'VCM', 26:'HCM', 27:'CHCM', 28:'RDWCV', 29:'RDWSD', 30:'Eritroblastos', 31:'Leuco', 32:'Neutro', 33:'ProMielo', 34:'Mielo', 35:'Metamielo', 36:'Bast', 37:'Segmentados', 38:'Eosino', 39:'Baso', 40:'Linfo', 41:'Mono', 42:'Plaq', 43:'VPM', 44:'PCR', 45:'Na', 46:'K', 47:'Cl', 48:'P', 49:'Mg', 50:'CaT', 51:'Cai', 52:'Ur', 53:'Creat', 54:'ProtTotal', 55:'Albumina', 56:'BT', 57:'BD', 58:'BI', 59:'TP', 60:'AP', 61:'INR', 62:'Fib', 63:'TTPA', 64:'R', 65:'TT', 66:'Ret', 67:'Retcor', 68:'Retabs', 69:'Eriglobal', 70:'IRF', 71:'Rethe', 72:'Ferritina', 73:'Tipagem', 74:'CD', 75:'TSH', 76:'T4L', 77:'T3', 78:'T4', 79:'VITD', 80:'LCRcels', 81:'LCRhem', 82:'LCRglic', 83:'LCRlact', 84:'LCRprot', 85:'HMCperi', 86:'UROC', 87:'FUNGOLCR', 88:'FUNGOURINA', 89:'LCRcult', 90:'Cariotipo', 91:'PCRCOVID', 92:'NSFENO', 93:'NSVANCO', 94:'U1pH', 95:'U1Densidade', 96:'U1Glic', 97:'U1PigBil', 98:'U1CorposCet', 99:'U1Sangue', 100:'U1Prot', 101:'U1Urobil', 102:'U1Nitrito', 103:'U1CelsEpitel', 104:'U1FilamentoDeMuco', 105:'U1LeucoPorCampo', 106:'U1EritroPorCampo', 107:'U1Morf', 108:'U1Cilindro', 109:'U1Cristais', 110: 'FosfAlc', 111: 'tgp', 112:'tgo', 113:'cai_mmol', 114:'cat_2', 115:'HMC_cateter', 116:'HMC_cateter_2', 117:'FA_2', 118: 'GGT', 119: 'TG', 120: 'Alfa1AntiT', 121:'adalcr', 122:'swabretal', 123:'ercretal', 124:'enterovancor', 125:'pesqfungolcr', 126:'urocsaco', 127:'hba1', 128:'hba2', 129:'hbf', 130:'ph_v', 131:'pco2_v', 132:'bic_v', 133:'be_v', 134:'lactato_v', 135: 'glicose_v', 136:'cai_v', 137:'pcr_2', 138:'pcradeno', 139:'hadvlcr', 140:'cmvlcr', 141:'ebvlcr', 142:'evlcr', 143:'parvolcr', 144:'herpeslcr', 145:'hhv6e7lcr', 146:'hpevlcr', 147:'vzvlcr', 148: 'vasadeno', 149: 'sarscovswab', 150: 'vasrev', 151: 'vasmetapneumo', 152: 'vasmycoplasma', 153: 'vasinfluenzaA', 154: 'vasinfluenzaB', 155: 'vaspara1', 156: 'vaspara2', 157: 'vaspara3', 158: 'vaspara4', 159: 'vasvsr', 160: 'vaschlamydia', 161: 'resclinda', 162: 'resoxa', 163: 'reseritro', 164: 'resgenta', 165: 'reslevo', 166: 'reslinez', 167: 'respeni', 168: 'resrifamp', 169: 'ressulfa', 170: 'resteico', 171: 'restigeci', 172: 'resvanco', 173: 'resertap', 174: 'esbl', 175: 'resnitrof', 176: 'resamica', 177: 'resampi', 178: 'rescefep', 179: 'rescefox', 180: 'resceftaz', 181: 'resceftriax', 182: 'rescefurox', 183: 'rescipro', 184: 'resimipenem', 185:'resmero', 186:'respipetazo', 187:'cov2_t', 188: 'hbs', 189: 'vdrlsangue', 190: 'vdrlsanguetitulo', 191: 'vdrllcr', 192: 'pcrtoxolcr', 193: 'toxoigg', 194: 'toxoigm', 195: 'antixa', 196: 'ckmb', 197: 'ck', 198: 'ddimero', 199: 'HBC', 200: 'troponina', 201: 'cortisol', 202: 'antiro', 203:'antila', 204: 'cmvurina', 205: 'antitg', 206: 'antitpo', 207: 'hco3bic', 208: 'sato2gaso', 209: 'glicgaso', 210: 'lactatogaso', 211: 'rdw2', 212: 'leuco_absoluto', 213: 'mielo_absoluto', 214: 'promielo_absoluto', 215: 'mielo_absoluto', 216: 'metamielo_absoluto', 217: 'bast_absoluto', 218: 'seg_absoluto', 219: 'linf_absoluto', 220: 'mono_absoluto', 221: 'eosino_absoluto', 222: 'baso_absoluto', 223: 'blastos', 224: 'plaq_2', 225:'nome'})
                    df = df.groupby(['RGHC', 'Date', 'Hour']).max()
                    df.to_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Output/Laboratorial - Sigma/' + elemRg + '.csv')

                    df_echo = df_echo.rename(columns={0:'RGHC', 1:'Data_1', 2:'Data_2', 3:'PCA', 4:'PCA_2', 5:'Ao', 6:'Ao_2', 7:'Ao_3', 8:'AE', 9:'AE_2', 10:'AE_3', 11:'VE', 12:'FE', 13:'CIV', 14:'VTI', 15:'PSAP'})

                    df_echo.to_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Output/Echo - Sigma/' + elemRg + '.csv')

                    df_usg = df_usg.rename(columns={0: 'RGHC', 1: 'Data_1', 2: 'Data_2', 3:'Hemorragia'})

                    df_usg.to_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Output/USG - Sigma/' + elemRg + '.csv')

            df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/Actual - CSV - Input - Algorithm/actual.csv', delimiter=';')
            df2 = df2[df2['rghc'].str.rstrip() != matricula]
            df2.to_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/Actual - CSV - Input - Algorithm/actual.csv', index=False, sep=';')
            
    except TimeoutException:
        browser.quit()
        continue

InvalidSessionIdException: Message: WebDriver session does not exist, or is not active
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:188:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.jsm:359:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.jsm:493:13
assert.session@chrome://remote/content/shared/webdriver/Assert.jsm:45:4
despatch@chrome://remote/content/marionette/server.js:301:19
execute@chrome://remote/content/marionette/server.js:277:16
onPacket/<@chrome://remote/content/marionette/server.js:250:20
onPacket@chrome://remote/content/marionette/server.js:251:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.js:502:20


In [9]:
df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/Actual - CSV - Input - Algorithm/actual.csv', delimiter=',')
#df2 = df2[df2['rghc'].str.rstrip() != matricula]
#df2.to_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/__Healthcare Data Science/Data Warehouse/Input/Laboratorial - Sigma/Actual - CSV - Input - Algorithm/actual.csv', index=False, sep=',')